In [1]:
import os
import re
import rasterio
from rasterio import Affine, MemoryFile
from rasterio.enums import Resampling

In [3]:

def create_folder(path):
    """
    Check if folder exists in current directory
    Create folder if folder does not exist
    """
    if not os.path.exists(path):
        os.makedirs(path)
        
# Adapted from user2856 answer https://gis.stackexchange.com/questions/329434/creating-an-in-memory-rasterio-dataset-from-numpy-array/329439#329439
def resample_raster(raster, scale, output_fname='default.tif'):
    """
    Function to resample raster and upscale by <scale> factor
    """
    t = raster.transform

    # rescale the metadata
    transform = Affine(t.a / scale, t.b, t.c, t.d, t.e / scale, t.f)
    height = int(raster.height * scale)
    width = int(raster.width * scale)

    profile = raster.profile
    profile.update(transform=transform, driver='GTiff', height=height, width=width)

    data = raster.read( # Note changed order of indexes, arrays are band, row, col order not row, col, band
            out_shape=(raster.count, height, width)
#             resampling=Resampling.bilinear, # removed resmapling method so we don't smooth the values
        )
    
    with rasterio.open(output_fname, 'w', **profile) as dst:
        dst.write(data.astype(rasterio.float32))

# Location where the original .tif images are stored
image_folder = 'layers'
upsampling_scale = 11132/30

# Iterate throughb image folders
for img_dir in os.listdir(image_folder):
    
    # Create new 'resampled folder' in image collection
    resampled_folder = os.path.join(image_folder, 'resampled')
    create_folder(resampled_folder)
    
    # Iterate through images and write resampled images to disk
    img_path = os.path.join(image_folder)
    tif_files = [f for f in os.listdir(img_path) if re.match('.*tif$', f)]
    print(f'Resampling {len(tif_files)} {img_dir} images...')
    
    for img in tif_files:
        infile = os.path.join(img_path, img)
        outfile = os.path.join(resampled_folder, img)
        with rasterio.open(infile) as src:
            resample_raster(src, upsampling_scale, outfile)
    print('...completed\n')


Resampling 14 bfte_precip_reclass.tif images...
...completed

Resampling 14 bfte_snowcov_reclass.tif images...
...completed

Resampling 14 bfte_snowden_reclass.tif images...
...completed

Resampling 14 bfte_snowdep_reclass.tif images...
...completed

Resampling 14 bfte_snowfall_reclass.tif images...
...completed

Resampling 14 bfte_snowmelt_reclass.tif images...
...completed

Resampling 14 bfte_surftemp_reclass.tif images...
...completed

Resampling 14 darcy_precip_reclass.tif images...
...completed

Resampling 14 darcy_snowcov_reclass.tif images...
...completed

Resampling 14 darcy_snowden_reclass.tif images...
...completed

Resampling 14 darcy_snowdep_reclass.tif images...
...completed

Resampling 14 darcy_snowfall_reclass.tif images...
...completed

Resampling 14 darcy_snowmelt_reclass.tif images...
...completed

Resampling 14 darcy_surftemp_reclass.tif images...
...completed

